<a href="https://colab.research.google.com/github/Jbenhur8901/Bellabeat-Devices-DataAnalysis/blob/main/E_commerce_Webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **About the project**


About

Hello, my name is Jacques Benhur ESSISSONGO, I am a BSc student in Data Science and certified in the Google Data analytics program. 

The only purpose of this project is to make available to the community of analysts and data scientists, a dataset of customer reviews on the largest companies from the Trustpilot website in order to perform analysis, build models and make recommendations.

This data is very useful as it contains a lot of information that can be used in different ways.

Customer reviews are comments given to a 
company based on a customer's experience 
with the organization. By obtaining and 
analyzing customer reviews, companies can 
measure customer satisfaction, identify 
recurring customer issues, determine areas for 
improvement in existing strategies, and even 
discover new trends that can be exploited.

### **Default Setting to run selenium on colab**

In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.2 MB 8.2 MB/s 
     |████████████████████████████████| 384 kB 51.5 MB/s 
     |████████████████████████████████| 140 kB 41.8 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 https://

### **importing the important librairies**


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

### **Defining the main classes that will perform the extraction**


In [12]:
class Trustpilot:

  def __init__(self,n_run=0):
    self.n_runs=n_run
    self.source_code=None

  def extract(self,ps):
    self.source_code = ps
    self.soup = BeautifulSoup(self.source_code,"html.parser")
    
    """the extration is done in two steps. First, it finds all the tags that meet the criteria and return it as a list.
    Secondly, it loops over all of the tags and extract the text in it""" 
    self.n_runs+=1 # incriment n_runs each time that the method is executed

    if self.n_runs==1:
      # if it is the first time that the method is executed

      #get the ratings
      all_ratings = self.soup.find_all("div",class_="star-rating_starRating__4rrcf star-rating_medium__iN6Ty")
      all_ratings.pop(0)
      self.ratings = [rate.find("img")["alt"] for rate in all_ratings]

      #get the usernames, number of reviews and locations
      all_profiles = self.soup.find_all("a", attrs={"name":"consumer-profile"})
      self.profiles=[profile.text for profile in all_profiles ]

      # get the dates
      all_dates = self.soup.find_all("p",class_="typography_body-m__xgxZ_ typography_appearance-default__AAY17 typography_color-black__5LYEn")
      self.dates = [ date.text for date in all_dates]

    # comments
      all_comments = self.soup.find_all("p",class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn")
      self.comments = [comment.text for comment in all_comments]
    else:
  
      all_ratings = self.soup.find_all("div",class_="star-rating_starRating__4rrcf star-rating_medium__iN6Ty")
      all_ratings.pop(0)
      all_profiles = self.soup.find_all("a", attrs={"name":"consumer-profile"})
      all_dates = self.soup.find_all("p",class_="typography_body-m__xgxZ_ typography_appearance-default__AAY17 typography_color-black__5LYEn")
      all_comments = self.soup.find_all("p",class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn")

      for rate,profile,date,comment in zip(all_ratings,all_profiles,all_dates,all_comments):
        
        self.ratings.append(rate.find("img")["alt"])
        self.profiles.append(profile.text)
        self.dates.append(date.text)
        self.comments.append(comment.text)

  def is_same_length(self):
    # check the length of all the arrays. If the arrays are not the same size return False
 
    to_check=[self.ratings,self.profiles,self.dates,self.comments]
    lengths = [len(self.ratings),len(self.profiles),len(self.dates),len(self.comments)]

    if max(lengths)==min(lengths):
      return True
    else:
      count1= lengths.count(max(lengths))
      count2= lengths.count(min(lengths))
      if count1<2:
        n = lengths.index(max(lengths))
        self.to_balance = to_check[n]
        self.difference = abs(max(lengths)-min(lengths))

      elif count2<2:
        n =  lengths.index(min(lengths))  
        self.to_balance=to_check[n]
        self.difference = abs(max(lengths)-min(lengths))
      return False

  def harmony(self):
    # make sure than all the arrays are the same lengths, else add nan

    if self.is_same_length():
      return True
    else:
      for missing in range(self.difference):
        self.to_balance.append(np.nan)

  def save(self):
    self.harmony()
    collected_data = {"profile":self.profiles,"date":self.dates,"rating out of 5":self.ratings,"comment":self.comments}
    self.df = pd.DataFrame(collected_data)
  
  def clean_df(self):
      df=self.df
      df["date"] = [date.split(":")[1] for index,date in df["date"].items()] # split at : and return only date
      df["rating out of 5"] = [rate.split(" ")[1] for index,rate in df["rating out of 5"].items()] # get only the rate
      countries= df["profile"].str.split("review",expand=True)[1].str.replace("s","")
      df.insert(1,"country",countries)

      df["date"] = pd.to_datetime(df["date"])
      df["rating out of 5"].astype("int")
      df.drop("profile",axis=1,inplace=True)
      self.cleaned_df=df
      self.cleaned_df.to_csv(f"trustpilot{self.n_runs}.csv")
      return self.cleaned_df

In [4]:

class RetailIndex:
  """This class extract data from retail index website"""
  def __init__(self,ps):
    self.ps = ps
    self.soup = BeautifulSoup(ps,"html.parser")
    self.extracted_data=[]
    self.n = 0
  def extract(self):
    
    # get all the tr tag with the specified attributes
    tr_tag = self.soup.find_all("tr",attrs={"height":"23","style":"height:17.0pt"})
    # get all the td tags and return a list
    td_tag=[BeautifulSoup(str(tag),"html.parser").find_all("td") for tag in tr_tag]
    # extract data from the td tags
    self.data=[tag.text for taglist in td_tag for tag in taglist]

  def split_list(self,level):
    # split a large list into smaller one base on the level provided
  
    self.extracted_data.append(self.data[self.n:self.n+level])
    self.n+=level
    if self.n<len(self.data):
      self.split_list(level)
  
  def save(self):

    import csv
    # saving the results into a csv file
    colname=["Rank","Company","Country","Main Sector(s)","2014","2015","2016","2017","2018","2019","2020","Online sales %"]
    with open("retail_index.csv","w") as f:
      file_csv = csv.writer(f)
      file_csv.writerow(colname)
      for row in self.extracted_data:
        file_csv.writerow(row)

    import pandas as pd
    self.df = pd.read_csv("retail_index.csv")
    self.df.set_index("Rank",inplace=True)

### **extracting data from retail index**

In [6]:
link1 = "https://www.retail-index.com/E-CommerceRetail.aspx"

driver = webdriver.Chrome(options=options)
driver.get(link1)
ps = driver.page_source # ps is the page source

extractor1 = RetailIndex(ps)
extractor1.extract()
extractor1.split_list(12)
extractor1.save()

driver.quit()

In [7]:
extractor1.df

,Company,Country,Main Sector(s),2014,2015,2016,2017,2018,2019,2020,Online sales %
Rank,,,,,,,,,,,
1,Amazon,USA,All Sectors,24.230,25.600,28.650,30.400,33.744,32.185,32.185,100%
2,Otto Group,Germany,Fashion,5.175,5.490,5.860,6.490,6.480,6.910,12.149,57%
3,Zalando,Germany,"Fashion, Footwear and Leather",2.214,2.958,3.640,4.119,5.388,6.483,6.483,100%
4,Apple,USA,Consumer Electronics,3.750,4.000,4.200,4.770,5.650,5.750,55.604,10%
5,Tesco,United Kingdom,Food/ All Sectors,3.533,4.350,4.100,4.250,4.500,4.800,53.830,9%
6,Veepee (Vente-Privée),France,Fashion,1.700,2.000,3.000,3.300,3.700,4.000,4.000,100%
7,Carrefour,France,Food/ All Sectors,1.800,1.850,1.860,2.278,2.800,3.100,55.764,6%
8,Ceconomy (Mediamarkt/Saturn),Germany,Consumer Electronics,1.500,1.766,1.952,2.407,2.592,2.935,21.455,14%
9,Bol.com (Ahold),Netherlands,Consumer Electronics/ All Sectors,680.000,900.000,1.300,1.600,2.100,2.800,2.800,100%


### **extracting data from trustpilot**

In [ ]:
#connect to the website with selenium
driver2 = webdriver.Chrome(options=options)
driver2.get("https://www.trustpilot.com/review/www.cdiscount.com?languages=all")
ps2 = driver2.page_source # get the page source

n=0
# instantiate Trustpilot class and extract the first page
extractor2 = Trustpilot()
extractor2.extract(ps2)

# mecanism to switch to next page 
anchor_has_href=True
while anchor_has_href:
  n+=1
  try:
    element = WebDriverWait(driver2, 10).until(EC.presence_of_element_located((By.LINK_TEXT,"Next page")))
    next_page = element.get_attribute("href")
    driver2.get(next_page)
    ps2 = driver2.page_source
    extractor2.extract(ps2)
 
  except Exception as e:
    anchor_has_href=False
    print(n)
    break
  finally:
    driver.quit()

In [36]:
extractor2.save()

In [37]:
extractor2.clean_df()

,country,date,rating out of 5,comment
0,FR,2022-11-07,1,"INADMISSIBLE,Nous avons acheté un frigo améric..."
1,BE,2022-11-08,1,Tout bonnement honteux ! Je passe commande le ...
2,FR,2022-11-08,3,bonjour dans l'ensemble je trouve ceux que je ...
3,FR,2022-10-25,1,Vente forcée ! J'ai opté pour un convertible e...
4,FR,2022-11-06,1,Franchement déçue de ce qu’est devenu cette pl...
...,...,...,...,...
6261,FR,2022-04-07,1,dommage qu'il ne soit pas fait mention de l'ab...
6262,FR,2022-04-07,5,navigation compliquée quand on sélectionne dif...
6263,FR,2022-04-07,5,SUPER TRES BIEN
6264,FR,2022-04-07,5,je trouve que que c est un peu la pagaille dan...


### **Reviews Data Cleaning**

In [153]:
def clean_df(dfs):
  df=dfs
  df["date"] = [date.split(":")[1] for index,date in df["date"].items()] # split at : and return only date
  df["rating out of 5"] = [rate.split(" ")[1] for index,rate in df["rating out of 5"].items()] # get only the rate
  countries= df["profile"].str.split("review",expand=True)[1].str.replace("s","")
  df.insert(1,"country",countries)

  df["date"] = pd.to_datetime(df["date"])
  df["rating out of 5"].astype("int")
  df.drop("profile",axis=1,inplace=True)
  return df

raw = pd.read_csv("amazon490.csv")
raw.drop("Unnamed: 0",axis=1,inplace=True)

In [154]:
extractor2.clean_df(raw)

,country,date,rating out of 5,comment
0,FR,2022-11-07,1,J'ai toujours le même problème. On me raccroch...
1,FR,2022-11-07,5,Je commande 1 à 3 fois par an. Tout s'est touj...
2,FR,2022-11-07,1,si je pouvias mettre 0 je le ferais incompéten...
3,FR,2022-11-07,1,"Terminé, je ne commanderais plus JAMAIS chez A..."
4,FR,2022-09-22,1,Cela fait des années que je suis cliente chez ...
...,...,...,...,...
9610,FR,2011-02-04,5,plusieurs commande également sur ce web marcha...
9611,FR,2010-11-29,5,Site qui centralise des vendeurs en ligne de n...
9612,FR,2010-04-28,5,"J'adore Amazon, fiable, rapide, sérieux, on tr..."
9613,NL,2010-01-07,4,Snel en goede service.
